In [1]:

# Set up the notebook
%pprint
import sys
if ('../py' not in sys.path): sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

# Load libraries
from FRVRS import nu, fu
import os.path as osp
from pandas import DataFrame
import pandas as pd


# Data Fixes for Metrics Evaluation Open World

In [3]:

# load data frames
data_frames_list = nu.load_data_frames(
    metrics_evaluation_open_world_df='', metrics_evaluation_open_world_file_stats_df='', metrics_evaluation_open_world_scene_stats_df=''
)
metrics_evaluation_open_world_df = data_frames_list['metrics_evaluation_open_world_df']
metrics_evaluation_open_world_file_stats_df = data_frames_list['metrics_evaluation_open_world_file_stats_df']
metrics_evaluation_open_world_scene_stats_df = data_frames_list['metrics_evaluation_open_world_scene_stats_df']

Attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/metrics_evaluation_open_world_df.pkl.
Attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/metrics_evaluation_open_world_file_stats_df.pkl.
Attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/metrics_evaluation_open_world_scene_stats_df.pkl.


In [4]:

logs_columns_set = set(metrics_evaluation_open_world_df.columns)
file_columns_set = set(metrics_evaluation_open_world_file_stats_df.columns)
intersection_columns = set(['file_name', 'logger_version', 'is_scene_aborted'])

# Drop the logs columns already recorded in the file and scene stats data frames
drop_columns = list(logs_columns_set.intersection(file_columns_set).intersection(intersection_columns))
if drop_columns:
    metrics_evaluation_open_world_df = metrics_evaluation_open_world_df.drop(columns=drop_columns)
    print(metrics_evaluation_open_world_df.shape) # (276926, 107)
    nu.store_objects(metrics_evaluation_open_world_df=metrics_evaluation_open_world_df)
    nu.save_data_frames(metrics_evaluation_open_world_df=metrics_evaluation_open_world_df)

(199476, 107)
Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/metrics_evaluation_open_world_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/metrics_evaluation_open_world_df.csv


In [5]:

logs_columns_set = set(metrics_evaluation_open_world_df.columns)
intersection_columns = set([
    'injury_record_injury_treated', 'injury_record_injury_treated_with_wrong_treatment', 'patient_demoted_health_level', 'patient_demoted_health_time_remaining',
    'patient_demoted_hearing', 'patient_hearing', 'patient_record_health_level', 'patient_record_hearing', 'player_location_left_hand_location',
    'player_location_right_hand_location'
])

# Drop the file stats columns that came with the process but are covered well enough in the logs data frame and add no value here
drop_columns = sorted(logs_columns_set.intersection(file_columns_set).intersection(intersection_columns))
if drop_columns:
    metrics_evaluation_open_world_file_stats_df = metrics_evaluation_open_world_file_stats_df.drop(columns=drop_columns)
    print(metrics_evaluation_open_world_file_stats_df.shape) # (86, 12)
    nu.store_objects(metrics_evaluation_open_world_file_stats_df=metrics_evaluation_open_world_file_stats_df)
    nu.save_data_frames(metrics_evaluation_open_world_file_stats_df=metrics_evaluation_open_world_file_stats_df)

In [6]:

file_columns_set = set(metrics_evaluation_open_world_file_stats_df.columns)
scene_columns_set = set(metrics_evaluation_open_world_scene_stats_df.columns)
intersection_columns = set(['logger_version'])

# Drop the scene columns already recorded in the file stats data frames
drop_columns = sorted(file_columns_set.intersection(scene_columns_set).intersection(intersection_columns))
if drop_columns:
    metrics_evaluation_open_world_scene_stats_df = metrics_evaluation_open_world_scene_stats_df.drop(columns=drop_columns)
    print(metrics_evaluation_open_world_scene_stats_df.shape) # (133, 48)
    nu.store_objects(metrics_evaluation_open_world_scene_stats_df=metrics_evaluation_open_world_scene_stats_df)
    nu.save_data_frames(metrics_evaluation_open_world_scene_stats_df=metrics_evaluation_open_world_scene_stats_df)

(76, 48)
Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/metrics_evaluation_open_world_scene_stats_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/metrics_evaluation_open_world_scene_stats_df.csv


In [7]:

# Remove duplicates from the file stats data frame
subset_columns = ['session_uuid']
mask_series = metrics_evaluation_open_world_file_stats_df.duplicated(subset=subset_columns)
if mask_series.any():
    metrics_evaluation_open_world_file_stats_df = metrics_evaluation_open_world_file_stats_df[~mask_series]
    print(metrics_evaluation_open_world_file_stats_df.shape)
    nu.store_objects(metrics_evaluation_open_world_file_stats_df=metrics_evaluation_open_world_file_stats_df)
    nu.save_data_frames(metrics_evaluation_open_world_file_stats_df=metrics_evaluation_open_world_file_stats_df)

In [8]:

# Remove duplicates from the scene stats data frame
subset_columns = ['session_uuid', 'scene_id']
mask_series = metrics_evaluation_open_world_scene_stats_df.duplicated(subset=subset_columns)
if mask_series.any():
    metrics_evaluation_open_world_scene_stats_df = metrics_evaluation_open_world_scene_stats_df[~mask_series]
    print(metrics_evaluation_open_world_scene_stats_df.shape)
    nu.store_objects(metrics_evaluation_open_world_scene_stats_df=metrics_evaluation_open_world_scene_stats_df)
    nu.save_data_frames(metrics_evaluation_open_world_scene_stats_df=metrics_evaluation_open_world_scene_stats_df)


## Provide OSU with Correctly Grouped Responder Type Stats

In [9]:

data_frames_list = nu.load_data_frames(first_responder_master_registry_file_stats_df='')
df = data_frames_list['first_responder_master_registry_file_stats_df']
mask_series = ~df.responder_category.isnull()
sorted(df[mask_series].responder_category.unique())

Attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/first_responder_master_registry_file_stats_df.pkl.


['EM Resident', 'EMT-Basic', 'Medical Student', 'Other', 'Paramedic']

In [10]:

# Add a column that correctly groups responder types
new_column_name = 'responder_category'
if (new_column_name in metrics_evaluation_open_world_file_stats_df.columns):
    metrics_evaluation_open_world_file_stats_df = metrics_evaluation_open_world_file_stats_df.drop(columns=[new_column_name])
    print(metrics_evaluation_open_world_file_stats_df.shape)
if (new_column_name not in metrics_evaluation_open_world_file_stats_df.columns):
    metrics_evaluation_open_world_file_stats_df[new_column_name] = 'EM Resident'
    
    # Store the results and show the new data frame shape
    nu.store_objects(metrics_evaluation_open_world_file_stats_df=metrics_evaluation_open_world_file_stats_df)
    nu.save_data_frames(metrics_evaluation_open_world_file_stats_df=metrics_evaluation_open_world_file_stats_df)
    print(metrics_evaluation_open_world_file_stats_df.shape) # (86, 13)

display(metrics_evaluation_open_world_file_stats_df.groupby(new_column_name).size().to_frame().rename(columns={0: 'record_count'}).sort_values(
    'record_count', ascending=False
).head(5))

Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/metrics_evaluation_open_world_file_stats_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/metrics_evaluation_open_world_file_stats_df.csv
(60, 3573)


,record_count
responder_category,
EM Resident,60


In [11]:

file_columns_set = set(metrics_evaluation_open_world_file_stats_df.columns)
intersection_columns = set(['overall_category', 'global_category', 'global_description', 'sub_category', 'sub_description', 'responder_type'])
sorted(file_columns_set.intersection(intersection_columns))

['responder_type']

In [12]:

# Add responder types subgrouping columns
groupby_columns = ['overall_category', 'global_category', 'global_description', 'sub_category', 'sub_description', 'responder_type']
if any(map(lambda x: x not in metrics_evaluation_open_world_file_stats_df.columns, groupby_columns)):
    file_path = '../data/xlsx/Responder_Categories_and_Counts_DPW.xlsx'
    dpw_responder_categories_df = pd.read_excel(file_path)

    # Get description data frame
    mask_series = dpw_responder_categories_df.isna().all(axis='columns')
    idx = dpw_responder_categories_df[mask_series].index.min()
    where_df = dpw_responder_categories_df.iloc[idx+1:]
    
    # Get categories data frame
    dpw_responder_categories_df = dpw_responder_categories_df.iloc[:idx].dropna(axis='columns', how='all')
    dpw_responder_categories_df.columns = ['overall_category', 'responder_type', 'global_category', 'sub_category', 'record_count']
    
    # Create global description column
    columns_list = ['global_category', 'global_description', 'record_count']
    df = where_df.iloc[:, 1:4].dropna(axis='index', how='all')
    df.columns = columns_list
    global_description_dict = df.set_index('global_category').global_description.to_dict()
    dpw_responder_categories_df['global_description'] = dpw_responder_categories_df.global_category.map(global_description_dict)
    
    # Create sub description column
    columns_list = ['sub_category', 'sub_description', 'record_count']
    df = where_df.iloc[:, 5:8].dropna(axis='index', how='all')
    df.columns = columns_list
    sub_description_dict = df.set_index('sub_category').sub_description.to_dict()
    dpw_responder_categories_df['sub_description'] = dpw_responder_categories_df.sub_category.map(sub_description_dict)
    
    # Add columns to file stats data frame
    df = dpw_responder_categories_df[groupby_columns].groupby(groupby_columns).size().reset_index(drop=False)
    assert not (df[0] != 1).any(), "You will have a problem with responder types"
    df = df.drop(columns=[0])
    assert not (df.groupby('responder_type').size() != 1).any(), "You have a problem with responder types"
    for (overall_category, global_category, global_description, sub_category, sub_description, responder_type), _ in df.groupby(groupby_columns):
        mask_series = (metrics_evaluation_open_world_file_stats_df.responder_type == responder_type)
        for cn in groupby_columns: metrics_evaluation_open_world_file_stats_df.loc[mask_series, cn] = eval(cn)
    
    # Store the results and show the new data frame shape
    nu.store_objects(metrics_evaluation_open_world_file_stats_df=metrics_evaluation_open_world_file_stats_df)
    nu.save_data_frames(metrics_evaluation_open_world_file_stats_df=metrics_evaluation_open_world_file_stats_df)
    print(metrics_evaluation_open_world_file_stats_df.shape) # (86, 18)

# overall_category	global_category	global_description	sub_category	sub_description	responder_type	record_count
# EM Resident	DOC	Physicians	DRS	Physician Resident	EM-RES1	15
display(metrics_evaluation_open_world_file_stats_df.groupby(groupby_columns).size().to_frame().rename(columns={0: 'record_count'}).sort_values(
    'record_count', ascending=False
).head(5))

Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/metrics_evaluation_open_world_file_stats_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/metrics_evaluation_open_world_file_stats_df.csv
(60, 3578)


,,,,,,record_count
overall_category,global_category,global_description,sub_category,sub_description,responder_type,
